In [ ]:
!pip install -qU transformers accelerate torch sentencepiece arxiv pandas tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 150.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 103.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 7.0 MB/s eta 0:00:00
   

In [ ]:
!pip uninstall torchvision -y
!pip install torchvision --index-url https://download.pytorch.org/whl/cu118

Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Looking in indexes: https://download.pytorch.org/whl/cu118
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 955.6/955.6 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 128.8 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.2.0
    Uninstalling triton-3.2.0:
      Successfully uninstalled triton-3.2.0
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.1
    Uninstalling sympy-1.13.1:
      Successfully uninstalled sympy-1.13.1
  Attempting uninstall: torch
    Found existing installation: torch 2.6.0+cu118
    Uninstalling torch-2.6.0+cu118:
      Successfully uninstalled torch-2.6.0+cu118
ERROR: pip'

In [ ]:
import arxiv
import pandas as pd
from tqdm import tqdm
from datetime import datetime, timedelta
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from huggingface_hub import login

Get only CS.CL papers from arxiv

In [ ]:
HF_TOKEN = "ACCESS_TOKEN_WITH_LLAMA_PERMISSION"
# (Get from https://huggingface.co/settings/tokens)
login(token=HF_TOKEN)

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
def get_papers():
    search = arxiv.Search(
        query="cat:cs.CL",
        max_results=5000,
        sort_by=arxiv.SortCriterion.SubmittedDate,
        sort_order=arxiv.SortOrder.Descending
    )

    papers = []
    for result in tqdm(search.results(), total=5000, desc="Fetching papers"):
        papers.append({
            'title': result.title,
            'abstract': result.summary,
            'published': result.published.strftime('%Y-%m-%d'),
            'authors': ', '.join([a.name for a in result.authors]),
            'pdf_url': result.pdf_url
        })
        if len(papers) >= 5000:
            break
    return papers

if __name__ == "__main__":
    papers = get_papers()
    df = pd.DataFrame(papers)
    df.to_csv("arxiv_cs_cl_papers.csv", index=False)
    print(f"Saved {len(df)} papers to arxiv_cs_cl_papers.csv")
    print("\nSample entry:")
    print(df.head(1).to_string(index=False))

Run the LLAMA model to generate the problem and solution from the abstract

In [ ]:
def extract_with_llama(text):
    prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
            You are a helpful assistant that extracts structured information from scientific research abstracts.
            Your task is to identify the main PROBLEM the research addresses and summarize the SOLUTION proposed.
            Always include technical details, methods, or techniques used in the SOLUTION if available.<|eot_id|>
            <|start_header_id|>user<|end_header_id|>
            Given the following abstract, extract and label the PROBLEM and the SOLUTION:

            Abstract:
            {text[:2000]}
            <|eot_id|><|start_header_id|>assistant<|end_header_id|>
            PROBLEM:"""

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    model.to(device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=400,
        temperature=0.3,
        do_sample=False,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id
    )
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    problem = full_response.split("PROBLEM:")[-1].split("SOLUTION:")[0].strip()
    solution = full_response.split("SOLUTION:")[-1].strip() if "SOLUTION:" in full_response else "N/A"
    return {"problem": problem, "solution": solution}


Generate dataset

In [ ]:
from tqdm import tqdm
import pandas as pd
from datetime import datetime

def create_dataset(papers):
    data = []
    for _, paper in tqdm(papers.iterrows(), total=len(papers), desc="Processing papers"):
        try:
            abstract = paper['abstract']
            title = paper['title']
            authors = paper['authors']
            date_str = paper['published']
            url = paper['pdf_url']

            authors_formatted = authors if isinstance(authors, str) else ', '.join(authors)
            date_formatted = datetime.strptime(date_str, '%Y-%m-%d').strftime('%Y-%m-%d')

            extracted = extract_with_llama(abstract)

            data.append({
                'title': title,
                'authors': authors_formatted,
                'date': date_formatted,
                'problem': extracted['problem'],
                'solution': extracted['solution'],
                'url': url
            })
        except Exception as e:
            print(f"Error processing {paper.get('title', 'unknown title')}: {str(e)}")
            continue
    return pd.DataFrame(data)


In [ ]:
papers = pd.read_csv("/content/arxiv_cs_cl_papers.csv")
df = create_dataset(papers)

csv_path = f"NLP_Problem_Solution_batch.csv"
df.to_csv(csv_path, index=False)
print(f"Saved {len(df)} papers to {csv_path}")
print("Sample output:")
print(df.head(1).to_string(index=False))